# Data Parsing and EDA (archieved)
--- 
This notebook archives a different method of parsing data, without aggregating the rows by transcript_id and transcript_position group. We believe that this method could generate better model, as it does not lose any information. However, it is also more computationally expensive. To save the model tuning time, we have decided to use the aggregated data for our model.

The following is what we have done (in order): 
1. Read in dataset0.json. 
2. Split the sequence into 5-mers and 1-mers, which creates 10 extra columns.
3. Since we were unsure if the standard deviation created during aggregation has played an important role in the model, we created the standard deviation columns for each of the columns.
4. We balanced and reduced the data by sampling 20 rows from each transcript_id and transcript_position group.
5. Merged result_df with the label data. The final merged_data df have 2,436,760 rows and 31 columns. 

In [2]:
import json
import pandas as pd

In [6]:
# unzip the data
!gzip -d data/dataset0.json.gz

'gzip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# load dataset0.json
data_list = []

with open('data/dataset0.json', 'r') as json_file:
    for line in json_file:
        data = json.loads(line)
        data_list.append(data)

In [5]:
# convert data_list to a pd.DataFrame
def explore_dataset(data_list):
    a_results = []
    i = 0
    for data in data_list:
        trans_id, first = next(iter(data.items()))
        position, second = next(iter(first.items()))
        sequence, data = next(iter(second.items()))
        fixed_features = [trans_id, position, sequence]
        for obs in data:
            a_results.append(fixed_features + obs)
        if (i % 10000 == 0):
            print(i)
        i += 1
    print("done")
    return a_results

results = explore_dataset(data_list)
result_df = pd.DataFrame(results)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
done


In [7]:
# rename columns
colnames = ['transcript_id', 'transcript_position', 'sequence']
for i in ['-1_flank', 'central', '+1_flank']:
    for j in ["length", "std", "mean"]:
        colnames.append(i + '_' + j)
print(colnames)
result_df.columns = colnames

['transcript_id', 'transcript_position', 'sequence', '-1_flank_length', '-1_flank_std', '-1_flank_mean', 'central_length', 'central_std', 'central_mean', '+1_flank_length', '+1_flank_std', '+1_flank_mean']


In [8]:
result_df.shape

(11027106, 12)

In [9]:
result_df['transcript_position'] = result_df['transcript_position'].astype(int)

In [10]:
result_df.head()

,transcript_id,transcript_position,sequence,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,+1_flank_std,+1_flank_mean
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2


In [ ]:
# List of features you want to calculate the standard deviation for (to compare with aggregated data)
features_to_std = ["-1_flank_length", "-1_flank_std", "-1_flank_mean", "central_length", "central_std", "central_mean", "+1_flank_length", "+1_flank_std", "+1_flank_mean"]

df = result_df.copy()
# Group by 'transcript_id' and 'transcript_position', then calculate the standard deviation
df[features_to_std] = result_df.groupby(['transcript_id', 'transcript_position'])[features_to_std].transform('std')

# We will merge df into result_df later, so we need to rename the columns
df.columns = ['transcript_id', 'transcript_position', 'sequence', '-1_flank_length_std',
       '-1_flank_std_std', '-1_flank_mean_std', 'central_length_std', 'central_std_std',
       'central_mean_std', '+1_flank_length_std', '+1_flank_std_std', '+1_flank_mean_std']

In [11]:
# Function to split a sequence into 1-mers and 5-mers
def split_into_kmers(sequence, k):
    kmers = [sequence[i:i + k] for i in range(len(sequence) - k + 1)]
    return kmers

# Split sequences into 1-mers and 5-mers
result_df['1-mer'] = result_df['sequence'].apply(lambda x: split_into_kmers(x, 1))
result_df['5-mer'] = result_df['sequence'].apply(lambda x: split_into_kmers(x, 5))

for i in range(7):
    column_name = '1-mer_' + str(i)
    result_df[column_name] = result_df['1-mer'].apply(lambda x: x[i])

for i in range(3):
    column_name = '5-mer_' + str(i)
    result_df[column_name] = result_df['5-mer'].apply(lambda x: x[i])

In [12]:
result_df.drop(['1-mer', '5-mer'], axis=1, inplace=True)

In [13]:
result_df.head()

,transcript_id,transcript_position,sequence,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,...,1-mer_0,1-mer_1,1-mer_2,1-mer_3,1-mer_4,1-mer_5,1-mer_6,5-mer_0,5-mer_1,5-mer_2
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,...,A,A,G,A,C,C,A,AAGAC,AGACC,GACCA
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,...,A,A,G,A,C,C,A,AAGAC,AGACC,GACCA
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,...,A,A,G,A,C,C,A,AAGAC,AGACC,GACCA
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,...,A,A,G,A,C,C,A,AAGAC,AGACC,GACCA
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,...,A,A,G,A,C,C,A,AAGAC,AGACC,GACCA


In [24]:
# Note that each group has at least 50 rows, we reduced it to 20 rows each group to make the dataset smaller and balance the classes
result_df.groupby(['transcript_id', 'transcript_position']).size().reset_index(name='count')

,transcript_id,transcript_position,count
0,ENST00000000233,244,185
1,ENST00000000233,261,172
2,ENST00000000233,316,185
3,ENST00000000233,332,200
4,ENST00000000233,368,198
...,...,...,...
121833,ENST00000641834,1348,73
121834,ENST00000641834,1429,69
121835,ENST00000641834,1531,64
121836,ENST00000641834,1537,57


In [ ]:
# sample only 20 rows per transcript_id and transcript_position group
def downsample_to_20_rows(group):
    if len(group) > 20:
        return group.sample(20, random_state=4266)  # Change random_state for reproducibility
    else:
        return group

# Apply the downsampling function
balance_result_df = result_df.groupby(['transcript_id', 'transcript_position']).apply(downsample_to_20_rows).reset_index(drop=True)

In [ ]:
# the balanced dataset took about 40 mins to generate, so we saved it to a csv file
file_path = "data/balanced_data.csv"
balance_result_df.to_csv(file_path, index=False)

In [13]:
# df is our aggregated standard deviation data, we want to merge it with the balanced dataset
df.drop(['sequence'], axis=1, inplace=True)
df = df.drop_duplicates()

In [ ]:
df.head()

In [11]:
new_merged_data = pd.merge(balance_result_df, df, on=['transcript_id', 'transcript_position'], how='inner')

In [ ]:
# save the merged data to a csv file, the file is 676MB, so we didn't upload it to github
new_merged_data.to_csv("data/std_merged_data.csv", index=False)

In [13]:
# The final merged_data df have 2,436,760 rows and 31 columns. 
new_merged_data.shape

(2436760, 31)